## Imports

In [ ]:
# Import DAGWorks
from dagworks import driver

# Import modules for Hamilton
from components import data_loader
from components import ftrs_autoregression
from components import ftrs_calendar
from components import ftrs_common_prep

# Some other imports
import os
from typing import Union

# Setting up config/keys

In [ ]:
def load_api_key():
    data = {}
    with open("../.dagworks") as f:
        for line in f.read().split("\n"):
            linesplit = line.split("=")
            if len(linesplit) != 2:
                continue
            key, value = linesplit
            data[key]=value
    if "API_KEY" in data:
        return data["API_KEY"]
    else:
        raise ValueError(f"Create a file in the base of the repo with the line API_KEY=... If you don't have one (or forgot), ")
config = {"schema_version" : "new"}
PROJECT_ID = 19306
USERNAME = "elijah@dagworks.io"
api_key = load_api_key()

# Instantiating the driver

In [ ]:
from hamilton import base
dr = driver.Driver(
        {},
        data_loader,
        ftrs_autoregression,
        ftrs_calendar,
        ftrs_common_prep,
        username="elijah@dagworks.io",
        api_key=api_key,
        project_id=PROJECT_ID,
        dag_name="time_series_processing",
        tags={"template": "time_series_processing", "iteration" : "1"},
        adapter=base.DefaultAdapter()
    )
dr.display_all_functions(graphviz_kwargs=dict(graph_attr={'size': '15,10'}))

# Executing the DAG

In [ ]:
all_possible_outputs = dr.list_available_variables()
desired_features = [
    o.name for o in all_possible_outputs
    if o.tags.get("stage") == "production"]

inputs = {"data_path": "data/train_sample.csv"}
result = dr.execute(desired_features, inputs=inputs)